Importando as Bibliotecas

In [26]:
import numpy as np
from scipy.special import gamma
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.interpolate import PchipInterpolator
import pandas as pd

#### Definindo Variáveis Iniciais

In [112]:
n = 10**6 # Número de vetores thetas
k = 10**1 # Número de bins
m = 3 # Dimensionalidade
x = np.random.randint(0, 11, m) # Vetor x
y = np.random.randint(0, 11, m) # Vetor y

alfa = x + y # Vetor Alfa = Vetor x + Vetor y

#### Definindo Funções Principais

In [4]:
def f(theta, alfa, constante):
    log_produtorio = np.sum((alfa - 1) * np.log(theta), axis=1) # Para acelerar as operações, faz-se log do produtório (vira somatório)
    return np.exp(log_produtorio)/constante

def beta_multivariavel(alfa): # Beta(a + b) = Gamma(a)Gamma(b)/Gamma(a + b)
    log_produtorio = np.sum(np.log(gamma(alfa)))
    return np.exp(log_produtorio)/gamma(np.sum(alfa))

#### Gerando Amostras por meio de MCMC

In [5]:
def f_dir(x, a):
    if ((x < 0).any()): return 0.0
    t=np.empty_like(x)
    for i in range(len(x)):
        t[i]=np.power(x[i],(a[i]-1))
    return np.prod(t)

def cria_cov(alfa):
    M = [0.0]*(m - 1)
    M = [M]*(m - 1)
    M = np.array(M)

    for i in range(m - 1):
        for j in range(m - 1):

            if i == j:
                M[i, j] = alfa[i] * (sum(alfa) - alfa[i]) / \
                          ((sum(alfa) ** 2) * (sum(alfa) + 1))  # cálculo das variâncias
            else:
                M[i, j] = -alfa[i] * alfa[j] / \
                      ((sum(alfa) ** 2) * (sum(alfa) + 1))  # cálculo das covariâncias

    return M

def met_ac(pontos,p,b,alfa):
    for i in range(1,len(pontos)):
        ponto_atual = [0]*m
        for j in range(m - 1):
            ponto_atual[j] = pontos[i-1][j] + p[i,j]
        ponto_atual[-1] = 1 - np.sum(ponto_atual[:-1])
        ponto_atual = np.array(ponto_atual)
        # algoritmo de aceitação de Metropolis
        ac = min(1, f_dir(ponto_atual, alfa) /
                 f_dir(pontos[i-1], alfa))
        if ac >= b[i]:
            pontos[i] = np.array(ponto_atual)
        else:
            pontos[i] = pontos[i-1]

    return pontos

def calcular_burn_in(cadeia, alfa, limiar_densidade=0.5):
    densidades = np.array([f_dir(theta, alfa) for theta in cadeia])
    max_densidade = np.max(densidades)
    limiar_alto = limiar_densidade * max_densidade

    max_index = 0
    max_density_value = 0

    for i, densidade in enumerate(densidades[:len(cadeia) // 4]):
        if densidade >= limiar_alto:
            return i
        if densidade > max_density_value:
            max_density_value = densidade
            max_index = i
    
    return max_index

def gera_dir(alfa, n):
    
    thetas = np.array([np.array([1]*m)/m for _ in range(n)])
    M = cria_cov(alfa)
    
    p = np.random.multivariate_normal([0]*(m - 1), M,size=n)  # gera da Normal Multivariada
    b = np.random.uniform(0,1,size=n)
    dir=met_ac(thetas,p,b,alfa)

    burn_in = calcular_burn_in(dir, alfa)
    return dir[burn_in:]

In [50]:
# Definindo a função f_nula
def f_nula(theta_1, x_1, x_2, x_3):
    theta_3 = (1 - theta_1**(1/2))**2
    theta_2 = 1 - theta_1 - theta_3

    return (theta_1[0]**x_1) * (theta_2[0]**x_2) * (theta_3[0]**x_3)/beta_multivariavel(alfa)

# Função a ser minimizada apenas em theta_1
def neg_f_nula(theta_1, x_1, x_2, x_3):
    value = -f_nula(theta_1, x_1, x_2, x_3)
    return value

def calcular_phi(vetor_x):
    # Minimizar a função com a restrição
    result = minimize(neg_f_nula, x0=0.5, bounds=[(0, 1)], method='Powell', args=(vetor_x[0], vetor_x[1], vetor_x[2]))  # x0 é o palpite inicial para theta_1

    # Ponto onde o máximo ocorre e o valor máximo de f_hardy
    theta_estrela_1 = result.x[0]
    phi = f_nula(np.array([theta_estrela_1]), vetor_x[0], vetor_x[1], vetor_x[2])

    # r(theta) = 1
    return phi

In [17]:
# Gerando x_1, x_2, x_3
n_hardy = 20
lista = []
for x_3 in range(2, 19):
    lista.append([1, n_hardy - 1 - x_3, x_3])

for x_3 in range(0, 11):
    lista.append([5, n_hardy - 5 - x_3, x_3])

for x_3 in range(0, 8):
    lista.append([9, n_hardy - 9 - x_3, x_3])

lista

[[1, 17, 2],
 [1, 16, 3],
 [1, 15, 4],
 [1, 14, 5],
 [1, 13, 6],
 [1, 12, 7],
 [1, 11, 8],
 [1, 10, 9],
 [1, 9, 10],
 [1, 8, 11],
 [1, 7, 12],
 [1, 6, 13],
 [1, 5, 14],
 [1, 4, 15],
 [1, 3, 16],
 [1, 2, 17],
 [1, 1, 18],
 [5, 15, 0],
 [5, 14, 1],
 [5, 13, 2],
 [5, 12, 3],
 [5, 11, 4],
 [5, 10, 5],
 [5, 9, 6],
 [5, 8, 7],
 [5, 7, 8],
 [5, 6, 9],
 [5, 5, 10],
 [9, 11, 0],
 [9, 10, 1],
 [9, 9, 2],
 [9, 8, 3],
 [9, 7, 4],
 [9, 6, 5],
 [9, 5, 6],
 [9, 4, 7]]

In [52]:
def criar_lista_v(lista_f_ord, n, k):
    # Define-se os pontos de corte v
    # - v_0 = 0; v_k = sup(f(theta))
    resto = n % k
    tamanho_bin = n // k

    # No entanto, quando n%k = c != 0, precisa-se adaptar o tamanho de c bins com um ponto a mais
    ajuste = np.concatenate((np.arange(1, resto + 1), np.array([resto] * (k - resto)))) # Array para ajustar o tamanho dos bins
    lista_intermed = np.arange(1, k + 1) * tamanho_bin + ajuste
    lista_v = np.concatenate(([0], lista_f_ord[lista_intermed - 1]))
    return lista_v, lista_intermed

def calcular_sev(ev, h=1, t=2):

    def Q(d, z):
        return stats.chi2.cdf(d, z)

    def Q_inv(d, c):
        return stats.chi2.ppf(c, d)

    def sigma(t, h, c):
        return Q(t - h, Q_inv(t, c))

    sev_unstandardized = sigma(t, h, ev)
    return 1 - sev_unstandardized

In [140]:
n = 10**3
k = 10

e_valores = []
for observacoes in lista:
    a_priori = np.array([0]*3)
    total = np.array(observacoes) + a_priori
    correcao = np.array([1]*3)
    # Cooreção pois a Dirichlet tem expoente alfa - 1 e a função posteriori de Hardy-Weinber tem expoente alfa

    alfa = np.array(total + correcao)
    s_estrela = calcular_phi(total)

    thetas = gera_dir(alfa, n)
    lista_f = f(thetas, alfa, beta_multivariavel(alfa))

    lista_f_ord = np.sort(lista_f)
    lista_v, lista_intermed = criar_lista_v(lista_f_ord, len(lista_f_ord), k)

    fracao = np.concatenate((np.array([0]), lista_intermed))/n
        
    interp = PchipInterpolator(lista_v, fracao)
    ev = interp(s_estrela)
    if ev > 1:
        ev = 1
    sev = calcular_sev(ev)
    e_valores.append([observacoes, ev, sev])

df1 = pd.DataFrame(e_valores)
df1.columns = ["x", "ev", "sev"]

e_valores = []
for observacoes in lista:
    a_priori = np.array([1]*3)
    total = np.array(observacoes) + a_priori
    correcao = np.array([1]*3)
    # Cooreção pois a Dirichlet tem expoente alfa - 1 e a função posteriori de Hardy-Weinber tem expoente alfa

    alfa = np.array(total + correcao)
    s_estrela = calcular_phi(total)

    thetas = gera_dir(alfa, n)
    lista_f = f(thetas, alfa, beta_multivariavel(alfa))

    lista_f_ord = np.sort(lista_f)
    lista_v, lista_intermed = criar_lista_v(lista_f_ord, len(lista_f_ord), k)

    fracao = np.concatenate((np.array([0]), lista_intermed))/n
        
    interp = PchipInterpolator(lista_v, fracao)
    ev = interp(s_estrela)
    if ev > 1:
        ev = 1
    sev = calcular_sev(ev)
    e_valores.append([observacoes, ev, sev])

df2 = pd.DataFrame(e_valores)
df2.columns = ["x", "ev", "sev"]

C:\Users\alexa\AppData\Local\Temp\ipykernel_7968\3036942976.py:3: RuntimeWarning: invalid value encountered in sqrt
  theta_3 = (1 - theta_1**(1/2))**2
